## Question 5

## Ridge Regression

It performs L2 regularization, by adding penalty term equivalent to squares of the magnitude of parameters.

### Implementing sgd functionality, based on the algorithm

1.  Initialise the parameters to ${w}^{(0)}$ and $\tau$ =1

2.  While a stopping condition is not met do:

    $\bullet$ randomly select a datapoint $({x_n},{y_n})$ from the training set
    
    $\bullet$ ${w}^{(\tau)}$ = ${w}^{(\tau - 1)} - {\eta}^{(\tau)} \nabla E_n({w}^{(\tau - 1)})$
    
    $\bullet$ $\tau$ = $\tau$ + 1

## 5.2 Using R (with no use of special libraries), implement an SGD algorithm

In [44]:
#importing the libraries
library(ggplot2)
library(reshape2)

In [45]:
# reading the data in csv file
read_data <- function(fname, sc) {
   data <- read.csv(file=fname,head=TRUE,sep=",")
   nr = dim(data)[1]
   nc = dim(data)[2]
   x = data[1:nr,1:(nc-1)]
   y = data[1:nr,nc]
   if (isTRUE(sc)) {
      x = scale(x)
      y = scale(y)
   }
   return (list("x" = x, "y" = y))
}


In [46]:
#defining predict function 
predict_func <- function(Phi, w){
    
    return(Phi%*%w)
} 


In [47]:
#calculating the error objective
train_obj_func <- function (Phi, w, label, lambda){
    
    # the L2 regulariser is already included in the objective function for training 
    return(mean((predict_func(Phi, w) - label)^2) + .5 * lambda * w %*% w)
}

In [48]:
#acquiring the errors of train and test data
get_errors <- function(train_data, test_data, W, lambda) {
    
   n_weights = dim(W)[1]
   errors = matrix(,nrow=n_weights, ncol=2)
   for (tau in 1:n_weights) {
      errors[tau,1] = train_obj_func(train_data$x, W[tau,],train_data$y, lambda)
      errors[tau,2] = train_obj_func(test_data$x, W[tau,],test_data$y, lambda)
   }
   return(errors)
}

In [49]:
sgd_train <- function(train_x, train_y, lambda, eta, epsilon, max_epoch) {

   train_len = dim(train_x)[1]
    
   # maximum termination condition
   tau_max = max_epoch * train_len 

   W <- matrix(,nrow=tau_max, ncol=ncol(train_x)) 
   W[1,] <- runif(ncol(train_x))
  
   tau = 1 # counter 
   obj_func_val <-matrix(,nrow=tau_max, ncol=1) 
   obj_func_val[tau,1] = train_obj_func(train_x, W[tau,],train_y, lambda)

   while (tau <= tau_max){

       # check termination criteria
       if (obj_func_val[tau,1]<=epsilon) {break}
 
       # shuffle data:
       train_index <- sample(1:train_len, train_len, replace = FALSE)
    
       # loop over each datapoint
       for (i in train_index) {
           # increment the counter
           tau <- tau + 1
           if (tau > tau_max) {break}

           # make the weight update
           y_pred <- predict_func(train_x[i,], W[tau-1,])
           W[tau,] <- sgd_update_weight(W[tau-1,], train_x[i,], train_y[i], y_pred, lambda, eta)

           # keep track of the objective funtion
           obj_func_val[tau,1] = train_obj_func(train_x, W[tau,],train_y, lambda)
       }
   }
   # resulting values for the training objective function as well as the weights
   return(list('vals'=obj_func_val,'W'=W))
}



In [50]:
# updating the weight vector for sgd
sgd_update_weight <- function(W_prev, x, y_true, y_pred, lambda, eta) {
   # MODIFY THIS FUNCTION FOr L2 REG
   grad = - (y_true-y_pred) * x 
   l2 <- lambda*W_prev
   return(W_prev - eta * (grad+l2))
}

In [51]:
play_with_sgd <- function() {
    
   #reading train and test data files
   dtrain = read_data("Task1C_train.csv", TRUE)
   dtest = read_data("Task1C_test.csv", TRUE)
    
   #initializing hyper parameters
   max_epoch = 20
   epsilon = .001
   eta = .01
   lambda=0

   train_res = sgd_train(dtrain$x, dtrain$y, lambda, eta, epsilon, max_epoch)
   errors = get_errors(dtrain, dtest, train_res$W, lambda)
   
}

In [52]:
play_with_sgd()

## Question 5.3

For each lambda in {0, 0.4, 0.8, …, 10}, build a regression model and
compute the training and testing errors, using the provided data
sets. While building each model, all parameter settings (initial
values, learning rate, etc) are exactly the same, except a lambda
value. Set the termination criterion as maximum of 20 x N weight
updates (where N is the number of training data). Create a plot of
error rates (use different colors for the training and testing errors),
where the x-axis is log lambda and y-axis is the error rate

In [53]:
play_with_sgd1 <- function() {
    
   #reading train and test data files
   sgd_train = read_data("Task1C_train.csv", TRUE)
   sgd_test = read_data("Task1C_test.csv", TRUE)
   
   #initializing hyper parameters
   N = 449
   max_epoch = 20
   epsilon = 0.001
   eta = .01
    
   #changing lambda values from 0 to 10, with increase of 0.4
   lambda_seq = seq(0,10,0.4)
   #creating an empty data frame that stores train and testing error.
   sgd_dataframe = data.frame('lambda'=1:length(lambda_seq),'train_error'=1:length(lambda_seq),'test_error'=1:length(lambda_seq))
   #building a regression model for different lambda values
   for (i in 1:length(lambda_seq)){
       train_values = sgd_train(sgd_train$x, sgd_train$y, lambda = lambda_seq[i], eta, epsilon, max_epoch)
       errors = get_errors(sgd_train, sgd_test, train_values$W, lambda = lambda_seq[i])
       sgd_dataframe[i,'lambda'] = lambda_seq[i]
       sgd_dataframe[i,'train_error'] = mean(errors[,1])
       sgd_dataframe[i,'test_error'] = mean(errors[,2])
    }
    
   #reshaping the data frame for visualization
   sgd_dataframe.m <- melt(sgd_dataframe, id='lambda')
   #giving the names for columns in a dataframe
   names(sgd_dataframe.m) <- c('lambda', 'typee', 'errorr')
   #plotting the graph which represents both training and testing errors
   graphh <- ggplot(data = sgd_dataframe.m, aes(x= log(lambda), y = errorr, color = factor(typee))) + geom_line() +  
        scale_color_discrete(guide = guide_legend(title = NULL)) + 
        ggtitle('Error rate vs. lambda') + theme_minimal()
   return (graphh) 
}
options(warn = -1) #ignore the warnings

In [55]:
#calling the sgd function
options(repr.plot.width = 10, repr.plot.height = 8)
play_with_sgd1()

### What’s the best value for lambda? Discuss lambda, model complexity, and error rates, corresponding to underfitting and overfitting?

1.The best value for lambda is always the value where the model do not suffer from neither over fitting nor under fitting.

2.If the lambda value is less, i.e the regularization parameter is less, the model becomes more complex and suffers from over-fitting.

3.If the lambda value is more, i.e the regularization parameter is more, the model becomes less complex and suffers from under-fitting.

4.From our graph, we can say that the best value of lambda could be something between 0 - 1.

5.Lower lambda value is having the lower error rate for both train and test errors and higher lambda values increase the train and test errors. So, we can conclude that with increase of lambda values the error rates of training and testing increases.

6.The difference between error rates of lambda between training and testing is high because, the model suffers from over-fitting for a smaller lambda values and learns the train data well and fail to generalize to a new unseen data. But, for larger values of lambda, as the model do not suffer from over-fitting, and try to identify the features of new unseen data, the error decreases.